In [164]:
import pandas as pd
import plotly
import pprint
import requests

### Generate unique filename

In [166]:
import os
data_folder = "temp"
filename = "temp_result_1.csv"

file_path = os.path.join(data_folder, filename)

filename = os.path.basename(file_path)
filesize = os.path.getsize(file_path)

import hashlib

def generate_unique_filename(filename, filesize):
    unique_string_filename = f"{filename}{filesize}"
    has_object = hashlib.md5(unique_string_filename.encode())
    hash_name = has_object.hexdigest()

    file_extension = filename.split(".")[-1]
    unique_string_filename = f"{hash_name}.{file_extension}"

    return unique_string_filename

unique_filename = generate_unique_filename(filename, filesize)
unique_filename

'0d86cf1b68704ab59cead7358f24ec9a.csv'

### read a csv

In [168]:
# Download a data file from Yahoo.com
path = "temp/temp_result_1.csv"
df = pd.read_csv(path)
df['Date'] = pd.to_datetime(df['Date'])
print(type(df))
df

<class 'pandas.core.frame.DataFrame'>


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-10-19,141.690002,144.949997,141.500000,143.860001,143.011597,61758300
1,2022-10-20,143.020004,145.889999,142.649994,143.389999,142.544373,64522000
2,2022-10-21,142.869995,147.850006,142.649994,147.270004,146.401489,86548600
3,2022-10-24,147.190002,150.229996,146.000000,149.449997,148.568619,75981900
4,2022-10-25,150.089996,152.490005,149.360001,152.339996,151.441589,74732300
...,...,...,...,...,...,...,...
245,2023-10-11,178.199997,179.850006,177.600006,179.800003,179.800003,47551100
246,2023-10-12,180.070007,182.339996,179.039993,180.710007,180.710007,56743100
247,2023-10-13,181.419998,181.929993,178.139999,178.850006,178.850006,51427100
248,2023-10-16,176.750000,179.080002,176.509995,178.720001,178.720001,52517000


In [35]:
import requests
import pandas as pd
from io import StringIO


df = pd.DataFrame(df)
# Convert the DataFrame to a CSV string
csv_string = df.to_csv(index=False)
# Define the FastAPI endpoint URL
upload_url = "http://localhost:8000/upload-dataframe/"  # Replace with your actual URL
# Create a file-like object from the CSV string
csv_file = StringIO(csv_string)
# Send a POST request to upload the DataFrame
files = {'file': (f'{unique_filename}.csv', csv_file, 'text/csv')}
response = requests.post(upload_url, files=files)
# Print the response
print(response.text)


{"message":"DataFrame uploaded and saved."}


### get Dataframe

In [22]:
import requests

# Update with the desired file name
filename = "data.csv"  

# Make a GET request to the endpoint
response = requests.get(f"http://localhost:8000/get-dataframe/?filename={filename}")

# Check the response status code
if response.status_code == 200:
    data = response.json()
    data = data["dataframe"]
    # Now you can work with the DataFrame in your Jupyter Notebook
    
else:
    print("Request failed with status code:", response.status_code)

pd.DataFrame(data).head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-10-19,141.690002,144.949997,141.500000,143.860001,143.011597,61758300
1,2022-10-20,143.020004,145.889999,142.649994,143.389999,142.544373,64522000
2,2022-10-21,142.869995,147.850006,142.649994,147.270004,146.401489,86548600
3,2022-10-24,147.190002,150.229996,146.000000,149.449997,148.568619,75981900
4,2022-10-25,150.089996,152.490005,149.360001,152.339996,151.441589,74732300


### Add model to Database

In [38]:
import requests
import pprint


# Define the URL of your FastAPI application
url = "http://localhost:8000/model/post_new_model"  # Replace with the actual URL of your FastAPI application
# Define the JSON data you want to send in the POST request
data = {
    "asset_name": "asset 1",
    "model_name": f"{unique_filename}",
    "market":     "US",
    "currency":   "USD"    
}
# Send a POST request with JSON data
response = requests.post(url, json=data)
if response.status_code == 200:
    response_data = response.json()
    print("Response:")
    pprint(response_data)
else:
    print("Request failed with status code:", response.status_code)


Response:
{'asset_name': 'asset 1',
 'currency': 'USD',
 'market': 'US',
 'model_name': '0d86cf1b68704ab59cead7358f24ec9a.csv'}


### Get all models

In [47]:
import requests
from pprint import pprint

# Define the URL of your FastAPI application
url = "http://localhost:8000/model/get_all_models"  # Replace with the actual URL of your FastAPI application

# Send a GET request to the / endpoint
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    data = response.json()
    pprint(data)

else:
    print("Request failed with status code:", response.status_code)


[]


In [44]:
import requests
from pprint import pprint

model_name = unique_filename
# Define the URL of your FastAPI application
url = f"http://127.0.0.1:8000/model/get_models_model_name/{model_name}"  # Replace with the actual URL of your FastAPI application

# Send a GET request to the / endpoint
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    data = response.json()
    pprint(data)

else:
    print("Request failed with status code:", response.status_code)


[{'asset_name': 'asset 1',
  'currency': 'USD',
  'id': 1,
  'market': 'US',
  'model_name': '0d86cf1b68704ab59cead7358f24ec9a.csv'}]


### update data


In [46]:
import requests

# Define the URL of your FastAPI application
id = 1
url = f"http://localhost:8000/model/delete_model/{id}"

# Send a Deleete request
response = requests.delete(url)

# Check the response status code
if response.status_code == 200:
    # Parse the JSON response and pretty-print it
    response_data = response.json()
    print("Response:")
    print(response_data)
else:
    print("Request failed with status code:", response.status_code)


Response:
{'message': 'Model ID: 1 is deleted successfully'}


### calculate NAV

In [56]:

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [58]:
def plotting_colors():
    while True:
        for color_choice in ['#DB4437', '#F4B400', '#0F9D58',
                  '#185ABC', '#B31412', '#EA8600', '#137333',
                  '#d2e3fc', '#ceead6']:
            yield color_choice



In [171]:
rf = pd.read_excel('temp/RF.xlsx')
rf.head(10)
rf = rf.iloc[5:]
rf.columns = ['Date', 'risk free rate']

rf['Date'] = pd.to_datetime(rf['Date'])
df = df.merge(rf, on='Date' )
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 0 to 126
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            127 non-null    datetime64[ns]
 1   Open            127 non-null    float64       
 2   High            127 non-null    float64       
 3   Low             127 non-null    float64       
 4   Close           127 non-null    float64       
 5   Adj Close       127 non-null    float64       
 6   Volume          127 non-null    int64         
 7   risk free rate  127 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 8.9+ KB


In [186]:
df1 = pd.read_csv("temp/MS.csv")
df1['Date'] = pd.to_datetime(df1['Date'])
df1 = df1.merge(rf, on='Date' )



In [225]:
def Cal_Drawdown(nav: pd.Series):
    return (nav / nav.expanding().max()-1)  # added missing parentheses

def Cal_nav(a_series):
    return a_series/a_series.iloc[0]*100

def sharpe_ratio(data):
    
    average_return = Cal_nav(data['Open']).rolling(window=1).mean()-100
    risk_free_rate = data['risk free rate']/100
    std_dev = Cal_nav(data['Open']).rolling(window=3).std().fillna(1)
    return (average_return - risk_free_rate) / std_dev



Portfolio_nav = Cal_nav(df['Open'])
drawdown = Cal_Drawdown(Portfolio_nav)
Portfolio_sharpe = sharpe_ratio(df)

benchmark = df1['Open']
Benchmark_nav = Cal_nav(benchmark)
Benchmark_drawdown = Cal_Drawdown(Benchmark_nav)
Benchmark_sharpe = sharpe_ratio(df1)


In [226]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,risk free rate
0,2022-10-19,141.690002,144.949997,141.500000,143.860001,143.011597,61758300,4.27757
1,2022-10-20,143.020004,145.889999,142.649994,143.389999,142.544373,64522000,4.32457
2,2022-10-21,142.869995,147.850006,142.649994,147.270004,146.401489,86548600,4.35843
3,2022-10-24,147.190002,150.229996,146.000000,149.449997,148.568619,75981900,4.32686
4,2022-10-25,150.089996,152.490005,149.360001,152.339996,151.441589,74732300,4.358


In [227]:
df1.head()

,Date,Open,High,Low,Close,Adj Close,Volume,risk free rate
0,2022-10-19,79.150002,79.589996,77.370003,77.639999,74.931038,7429600,4.27757
1,2022-10-20,77.629997,78.540001,76.139999,76.599998,73.927330,7638100,4.32457
2,2022-10-21,76.610001,79.529999,76.220001,79.220001,76.455910,8149000,4.35843
3,2022-10-24,79.529999,81.099998,78.570000,79.430000,76.658585,8143300,4.32686
4,2022-10-25,78.610001,80.669998,78.610001,80.510002,77.700912,8173600,4.358


In [231]:
fig = make_subplots(rows=3, cols=1, 
                    subplot_titles=("NAV", "Max Drawdown", "Sharpe Ratio" ), vertical_spacing=0.1)
                    
fig.update_layout(title_text="Portfolio overview", 
                  title_x=0.5, 
                  title_xanchor="center", 
                  title_xref="paper", 
                  height=800, 
                  width=700,
                  font=dict(family="Consolas")
                  )  
                  
# add NAV plot
set_Date_as_x = df['Date']

# Portfolio NAV
fig.add_scatter(x=set_Date_as_x, y=Portfolio_nav, name=f"Portfolio NAV",
                legendgroup="Portfolio", legendgrouptitle_text="Portfolio", row=1, col=1, 
                line=dict(color="#4285F4", width=2))

fig.add_scatter(x=set_Date_as_x, y=Benchmark_nav, name=f"Benchmark NAV",
                legendgroup="Benchmark", legendgrouptitle_text="Benchmark", row=1, col=1, 
                line=dict(color="darkviolet", width=2))
# Portfolio Drawdown
fig.add_scatter(x=set_Date_as_x, y=drawdown, name=f"Portfolio Drawdown",
                legendgroup="Portfolio", legendgrouptitle_text="Drawdown", row=2, col=1, 
                line=dict(color="#4285F4", width=2))

fig.add_scatter(x=set_Date_as_x, y=Benchmark_drawdown, name=f"Benchmark_Drawdown",
                legendgroup="Benchmark", legendgrouptitle_text="Benchmark_Drawdown", row=2, col=1, 
                line=dict(color="darkviolet", width=2))
# Portfolio Sharpe
fig.add_scatter(x=set_Date_as_x, y=Portfolio_sharpe, name=f"Portfolio_Sharpe",
                legendgroup="Portfolio", legendgrouptitle_text="Portfolio_sharpe", row=3, col=1, 
                line=dict(color="#4285F4", width=2))

fig.add_scatter(x=set_Date_as_x, y=Benchmark_sharpe, name=f"Benchmark_Sharpe",
                legendgroup="Benchmark", legendgrouptitle_text="Benchmark_sharpe", row=3, col=1, 
                line=dict(color="darkviolet", width=2))
fig.show()